In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
import numpy as np
from pydrake.all import (
    RigidTransform,
    RotationMatrix,
    StartMeshcat,
    RandomGenerator,
    Diagram,
)
import numpy as np

import sponana.utils
from sponana.planner.rrt import SpotProblem, rrt_planning
from sponana.planner.navigator import Navigator
from sponana.planner.utils import visualize_path
from sponana.planner.rrt_tools import calc_intermediate_qs_wo_collision

import sponana.sim

In [4]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [ ]:
# Clean up the Meshcat instance.
rng = np.random.default_rng(145)  # this is for python
generator = RandomGenerator(rng.integers(0, 1000))  # this is for c++

# simulation_time = -1  # run indefinitely until ESC is pressed
simulation_time = 1
add_debug_logger = True
add_fixed_cameras = False
use_teleop = False

simulator, diagram = sponana.sim.clutter_gen(
    meshcat,
    rng,
    add_debug_logger=add_debug_logger,
    simulation_time=simulation_time,
    add_fixed_cameras=add_fixed_cameras,
    use_teleop=use_teleop,
)

In [5]:
q_start = np.array([3.0, 7.0, -1.57])
q_goal = np.array([1.0, 4.0, -3.13316598])

In [8]:
navigator = Navigator()
spot_problem = SpotProblem(
    q_start=q_start, q_goal=q_goal, collision_checker=navigator._collision_check
)
path = rrt_planning(spot_problem, 1500, 0.05)

In [18]:
print(len(path))

visualize_path(path, meshcat)

249


In [ ]:
sponana.utils.visualize_diagram(diagram, 1)

In [13]:
from sponana.hardcoded_cameras import get_base_positions_for_hardcoded_cameras

target_base_positions = get_base_positions_for_hardcoded_cameras()
visualize_path(target_base_positions.reshape(-1, 3))
target_base_positions

array([[[ 1.00000000e+00,  4.00000000e+00, -3.13316598e+00],
        [-6.96709467e-01,  3.15240161e+00,  8.57867299e-01],
        [-6.03719954e-01,  4.85234302e+00, -9.63954086e-01]],

       [[ 1.00000000e+00,  1.82786728e-12, -3.13316598e+00],
        [-6.96709467e-01, -8.47598387e-01,  8.57867299e-01],
        [-6.03719954e-01,  8.52343020e-01, -9.63954086e-01]],

       [[ 1.00000000e+00, -4.00000000e+00, -3.13316598e+00],
        [-6.96709467e-01, -4.84759839e+00,  8.57867299e-01],
        [-6.03719954e-01, -3.14765698e+00, -9.63954086e-01]]])

In [ ]:
def check_straight_line_shortcutting(node1, node2):
    spot_problem = SpotProblem(node1, node2, self._collision_check)
    rrt_tools = RRT_tools(spot_problem)
    q_goal = problem.goal
    straight_path = calc_intermediate_qs_wo_collision(node1, node2)
    if straight_path[-1] == node2:
        return True
    else:
        return False

In [ ]:
def two_nodes_shortcutting(path):
    #https://www.cs.cmu.edu/~maxim/classes/robotplanning/lectures/RRT_16350_sp23.pdf
    n0_ind = 0 # start
    n1_ind = n0_ind+1
    new_path = []
    goal_node = path[-1]
    goal_node_ind = len(path)-1
    while path[n0_ind] != goal_node:
        n0 = path[n0_ind]
        n1 = path[n1_ind]
        while check_straight_line_shortcutting(n0, n1) and (n1_ind+1) < goal_node_ind:
            n1_ind += 1
        new_path.append(n0, n1)
        n0_ind = n1_ind
        n1_ind = n1_ind + 1 
